# Documentación de Scrapers

Este notebook contiene la documentación y ejemplos de uso de los componentes de scraping y construcción de modelos de datos disponibles en la carpeta `src/scrapers`.

## Índice

1. [Scraping de Páginas (Deep Scraper)](#scraping-de-paginas)
   - Scraping de una sola página (`page_scraper`)
   - Crawling recursivo (`page_deep_scraper`)
2. [Construcción de Modelos (Model Builder)](#construccion-de-modelos)
   - Identidad de la compañía (`data_to_identity`)
   - Generación del modelo completo (`from_url_model`)
3. [Utilidades de Extracción](#utilidades-de-extraccion)
   - Overview de módulos de utilidad

4. [Scraping de Perfiles Externos](#scraping-de-perfiles-externos)
   - Extracción de TheCrowdSpace (`thecrowdspace_profile_scraper`)
5. [Scrapers Específicos](#scrapers-especificos)
   - Extracción de Favicon (`get_favicon_url`)
---

*Nota: Este documento debe actualizarse a medida que se agreguen nuevas funcionalidades al módulo de scrapers.*

## ⚙️ Configuración Notebook
Define el PATH para el correcto funcionamiento del notebook y la importación de módulos.

In [ ]:
# Se asume que el paquete 'src' esta instalado en el entorno (pip install -e .)
import sys
print(f"✅ Python executable: {sys.executable}")


---
## 1. Scraping de Páginas (Deep Scraper) <a name="scraping-de-paginas"></a>

El módulo `page_deep_scraper.py` se encarga de la extracción de contenido web. Funciona en dos niveles:
1. **Nivel Página**: Extrae links y textos de una URL específica, segmentando por `head`, `header`, `main` y `footer`.
2. **Nivel Sitio (Crawler)**: Navega recursivamente por un dominio (BFS) para mapear su estructura.

In [ ]:
# Importar funciones de scraping
from src.scrapers.page_deep_scraper import page_scraper, page_deep_scraper
import json

# Función auxiliar para imprimir JSON bonito
def print_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))

### `page_scraper(url_base)`

Extrae contenido de **una sola URL**.

**Retorna:** Un objeto de página con:
- `page`: La URL procesada.
- `links`: Diccionario con listas de URLs encontradas en `head`, `header`, `main`, `footer`.
- `texts`: Diccionario con listas de textos encontrados en las mismas secciones.

In [ ]:
# Ejemplo: Scrapear una sola página
TEST_URL = "https://fraccional.cl"

try:
    page_data = await page_scraper(TEST_URL)
    
    print(f"✅ Página scrapeada: {page_data['page']}")
    print("\nSecciones con links:")
    for section, links in page_data['links'].items():
        print(f"  - {section}: {len(links)} links")
        
    print("\nSecciones con textos:")
    for section, texts in page_data['texts'].items():
        print(f"  - {section}: {len(texts)} textos")
        
except Exception as e:
    print(f"❌ Error: {e}")

### `page_deep_scraper(url_base, max_pages=100)`

Realiza un **crawl recursivo** (Breadth-First Search) comenzando desde `url_base`.

**Características:**
- Respeta el **dominio raíz** (no sale del sitio).
- Detecta y evita ciclos (visita cada URL una sola vez).
- Limita la cantidad de páginas visitadas con `max_pages`.
- Captura errores por página sin detener el proceso global.

**Retorna:** Un objeto resumen con:
- `rootDomain`: El dominio base.
- `pagesScraped`: Cantidad total de páginas procesadas.
- `allInternalLinks`: Todos los links internos únicos encontrados.
- `pages`: Diccionario con el detalle de cada página (resultado de `page_scraper`).

In [ ]:
# Ejemplo: Deep Scraper (limitado a 3 páginas para demo)
try:
    deep_data = await page_deep_scraper(TEST_URL, max_pages=50)
    
    print(f"🌎 Root Domain: {deep_data['rootDomain']}")
    print(f"📄 Páginas scrapeadas: {deep_data['pagesScraped']}")
    print(f"🔗 Total links internos únicos: {len(deep_data['allInternalLinks'])}")
    print("\nURLs visitadas:")
    for url in deep_data['pages'].keys():
        print(f"  - {url}")
        
except Exception as e:
    print(f"❌ Error: {e}")

---
## 2. Construcción de Modelos (Model Builder) <a name="construccion-de-modelos"></a>

El módulo `model_builder.py` transforma la data "cruda" del scraper en un modelo estructurado (Company Model) listo para ser almacenado en la base de datos.

In [ ]:
from src.scrapers.model_builder import data_to_identity, from_url_model, page_deep_scraped_to_dataSources

### `data_to_identity(url, name, slug, primary_domain)`

Normaliza y genera la identidad básica de una compañía (slug, name, primary_domain).

**Reglas:**
- `primaryDomain`: Se normaliza usando `tldextract` y se limpia de `www.`.
- `slug`: Se genera a partir del dominio si no se provee. Se reemplazan espacios por guiones.
- `name`: Se genera a partir del slug si no se provee (Capitalizado).

In [ ]:
identity = data_to_identity(
    url="https://www.ejemplo.negocio.com/inicio",
    name=None, # Auto-generar
    slug=None  # Auto-generar
)

print("🆔 Identidad Generada:")
print_json(identity)

### `from_url_model(url, name, slug, primary_domain)`

Es la función de alto nivel que orquesta todo el proceso:
1. Genera la identidad.
2. Ejecuta `page_deep_scraper`.
3. Convierte el resultado en una lista de `dataSources` estandarizada.
4. Retorna el objeto completo de la compañía.

**Estructura del Output:**
```json
{
  "slug": "...",
  "name": "...",
  "primaryDomain": "...",
  "dataSources": [
     { "url": "...", "links": {...}, "texts": {...} },
     ...
  ]
}
```

In [ ]:
# Ejemplo: Crear modelo completo desde una URL (con límite para demo)
# Nota: from_url_model llama a page_deep_scraper internamente.
# Para este ejemplo, simularemos una llamada rápida.

print("🚀 Iniciando construcción del modelo (esto puede tardar unos segundos)...")

try:
    company_model = await from_url_model(url=TEST_URL)
    
    print("✅ Modelo construido exitosamente:")
    print(f"  Nombre: {company_model['name']}")
    print(f"  Slug: {company_model['slug']}")
    print(f"  DataSources: {len(company_model['dataSources'])}")
    
    # Ver un dataSource de ejemplo
    if company_model['dataSources']:
        print("\nEjemplo de DataSource[0]:")
        first_ds = company_model['dataSources'][0]
        print(f"  URL: {first_ds['url']}")
        print(f"  Links Head: {len(first_ds.get('links', {}).get('head', []))}")

except Exception as e:
    print(f"❌ Error: {e}")

---
## 3. Utilidades de Extracción <a name="utilidades-de-extraccion"></a>

La carpeta `src/scrapers/utils` contiene herramientas de bajo nivel usadas por los scrapers. Estas pueden ser usadas independientemente para tareas específicas o debugging.

### Componentes Principales:

- **`requestHTTP.py`**: Manejo de peticiones HTTP con `aiohttp` y `playwright` (en el futuro).
- **`html_spliter...py`**: Divide el HTML crudo en secciones semánticas (`head`, `header`, `main`, `footer`).
- **`urls_extractor...py`**: Extrae todas las URLs crudas de un string HTML.
- **`urls_utilities_cleaner.py`**: Limpia URLs (quita espacios, caracteres inválidos).
- **`text_extractor...py`**: Extrae texto visible limpio del HTML.


In [ ]:
# Ejemplo: Uso directo de utilidades para debug
from src.scrapers.utils.requestHTTP import fetch_html
from src.scrapers.utils.html_spliter_head_header_main_footer import html_spliter_head_header_main_footer

try:
    # 1. Fetch HTML manual
    raw_html = await fetch_html(TEST_URL)
    print(f"HTML descargado: {len(raw_html)} caracteres")
    
    # 2. Split HTML
    sections = html_spliter_head_header_main_footer(raw_html)
    print("Tamaño de secciones:")
    for sec, content in sections.items():
        print(f"  {sec}: {len(content)} caracteres")
        
except Exception as e:
    print(f"Error: {e}")

---
## 4. Scraping de Perfiles Externos <a name="scraping-de-perfiles-externos"></a>

Este módulo permite la extracción de información detallada desde perfiles de terceros, específicamente de **TheCrowdSpace**.

In [ ]:
from src.scrapers.thecrowdspace_profile_scraper import thecrowdspace_profile_scraper

### `thecrowdspace_profile_scraper(url, html_content=None)`

Extrae estructuradamente data de un perfil público de TheCrowdSpace.

**Argumentos:**
- `url`: URL del perfil.
- `html_content`: (Opcional) HTML crudo si ya fue descargado previamente.

**Retorna:** Un diccionario completo que coincide directamente con el campo `theCrowdSpace` del modelo de las compañías, con secciones:
- `hero`: Logo, verificación, industrias.
- `content`: Tarjetas superiores y descripciones largas.
- `sidebar`: Estadísticas, región operativa, links sociales, estado, etc.
- `p2p`: Scores y análisis de riesgos.
- `team`: Lista de miembros y cargos.

In [ ]:
# Ejemplo: Scrapear un perfil de TheCrowdSpace
CROWDSPACE_URL = "https://thecrowdspace.com/platform/bergfurst"

try:
    print(f"🕵️‍♀️ Scrapeando: {CROWDSPACE_URL} ...")
    # Nota: Si devuelve vacío puede ser por bloqueo de bot o URL inválida.
    # Se recomienda usar headers reales o un proxy en producción.
    profile_data = thecrowdspace_profile_scraper(CROWDSPACE_URL)
    
    if profile_data:
        print("✅ Data extraída exitosamente!")
        
        print(f"\n🏢 Nombre/Web detectada para: {profile_data.get('theCrowdSpaceUrl')}")
        
        if 'hero' in profile_data:
             print(f"  Industrias: {profile_data['hero'].get('industries')}")
             
        if 'sidebar' in profile_data:
             sb = profile_data['sidebar']
             print(f"  Website: {sb.get('websiteUrl')}")
             print(f"  Status: {sb.get('status')}")
             if 'operatesIn' in sb:
                 print(f"  Operates In: {sb['operatesIn'].get('countries')}")
        
        if 'team' in profile_data:
             print(f"  Team Members: {len(profile_data['team'])}")
             
    else:
        print("⚠️ No se pudo extraer información o el perfil está vacío.")
        
except Exception as e:
    print(f"❌ Error: {e}")

---
## 5. Scrapers Específicos <a name="scrapers-especificos"></a>

Scrapers diseñados para tareas puntuales de extracción de recursos.

In [ ]:
from src.scrapers.favicon_scraper import get_favicon_url

### `get_favicon_url(url)`

Busca y valida el favicon de una URL dada. Implementa heurísticas robustas:
1. Busca tags `<link rel="icon" ...>` en el HTML.
2. Manejo de rutas relativas y absolutas.
3. Fallback a `/favicon.ico` en la raíz del dominio.
4. Validación final por status HTTP 200.

**Retorna:** String con la URL absoluta del favicon o lanza Exception si falla.

In [ ]:
# Ejemplo: Obtener favicon
TARGET_URL = "https://google.com"

try:
    favicon = get_favicon_url(TARGET_URL)
    print(f"✅ Favicon encontrado: {favicon}")
except Exception as e:
    print(f"❌ Error: {e}")

---
## Extensibilidad

Para agregar nuevos scrapers o funcionalidades:

1. **Nuevos Parsers**: Agregarlos en `src/scrapers/utils/` si son utilidades genéricas de limpieza o extracción.
2. **Lógica de Scraping**: Modificar `src/scrapers/page_deep_scraper.py` para mejorar la navegación o la extracción de datos específica.
3. **Modelado**: Actualizar `src/scrapers/model_builder.py` si cambia la estructura de la base de datos (DataSources).

Consulte `src/scrapers/estructura_recomendada.txt` para ver la estructura modular planificada para el futuro.